In [1]:
import pandas as pd
import numpy as np

# for the model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, GRU, RepeatVector, TimeDistributed
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
# load files from pre-processing

X = np.load('X.npy') #rainpredictions, hour of day, month of year, holiday, ones
y = np.load('y.npy')
TimeHour = pd.read_csv('TimeHour.csv')

In [3]:
# split dataset in test and train

def ReshapeXData(dataframe):
    n_hours = 24
    n_features = dataframe.shape[1]
    days = int(dataframe.shape[0]/24)
    dataframe_shaped = np.reshape(dataframe,(days, n_hours, n_features))
    return dataframe_shaped

def ReshapeYData(dataframe):
    n_hours = 24
    n_features = 1
    days = int(dataframe.shape[0]/24)
    dataframe_shaped = np.reshape(dataframe,(days, n_hours, n_features))
    return dataframe_shaped

X_train_gru = X[0:10800,:]
X_test_gru = X[10801:13609,:]
Y_train_gru = y[0:10800]
Y_test_gru = y[10801:13609]


X_train_gru_shaped=ReshapeXData(X_train_gru)
X_test_gru_shaped = ReshapeXData(X_test_gru)
Y_train_gru_shaped = ReshapeYData(Y_train_gru)
Y_test_gru_shaped = ReshapeYData(Y_test_gru)
X_shaped = ReshapeXData(X[0:13608,:])
Y_shaped = ReshapeYData(y[0:13608])

In [4]:
# basic model

from keras.layers import Dense

lr=0.03
epochs=100
batch_size=50
validation_split=0.1

print('Build model...')

n_timesteps, n_features, n_outputs = X_train_gru_shaped.shape[1], X_train_gru_shaped.shape[2], Y_train_gru_shaped.shape[1]
# reshape output into [samples, timesteps, features]

model = Sequential()
model.add(GRU(200, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(RepeatVector(n_outputs))
model.add(GRU(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(100, activation='relu')))
model.add(TimeDistributed(Dense(1)))
model.compile(loss='mae', optimizer='adam')

model.fit(
        X_train_gru_shaped,
        Y_train_gru_shaped,
        epochs=epochs,
        batch_size=batch_size,
        verbose=1,
        validation_split=validation_split,
        callbacks=[
            ReduceLROnPlateau(
                verbose=1)])
    
    
#model = Sequential()
#model.add(Dense(1, input_dim=X.shape[1]))
#model.compile(optimizer = optimizers.Adam(lr=lr), loss='mean_squared_error')
#model.fit(X_train, y_train,
#          validation_split=validation_split,
#          epochs=epochs,
#          batch_size=batch_size,
#          shuffle=False)

Build model...
Train on 405 samples, validate on 45 samples
Epoch 1/150
405/405 [==============================] - 1s 3ms/step - loss: 134.8607 - val_loss: 102.7180
Epoch 2/150
405/405 [==============================] - 1s 1ms/step - loss: 123.3655 - val_loss: 63.7328

In [5]:
# make a prediction to test how well the model preforms
y_predicted = model.predict(X_shaped)

In [6]:
### LZS: WE HAVE TO IMPLEMENT HERE SOME METRICS AND A LOT OF PLOTS SO THAT WE CAN SEE HOW WELL THE MODEL PERFORMS
### LZS: AND ALSO TO BE ABLE TO FIND NEW FEATURES TO IMPROVE THE MODEL
### LZS: AND TO EVALUATE IF THE MODEL INDEED IMPROVES WHEN ADDING MORE FEATURES/ OTHER LAYERS

In [7]:
# show the inputs, predicted outputs and real outputs
for i in range(len(X_shaped[0])):
        for j in range(len(X_shaped[1])):
            print("Predicted=%s, Original=%s" % (y_predicted[i][j], Y_shaped[i][j]))

Predicted=[309.35263], Original=[504.06447306]
Predicted=[422.32217], Original=[502.99539014]
Predicted=[437.19647], Original=[501.56176333]
Predicted=[427.0708], Original=[267.72203709]
Predicted=[418.52634], Original=[164.6846228]
Predicted=[415.77573], Original=[415.60606761]
Predicted=[415.85388], Original=[506.92554056]
Predicted=[416.4615], Original=[498.48488889]
Predicted=[416.81702], Original=[539.77078944]
Predicted=[416.92993], Original=[569.86896236]
Predicted=[416.95096], Original=[573.43357472]
Predicted=[416.96265], Original=[554.18056333]
Predicted=[416.98297], Original=[519.03118042]
Predicted=[417.00647], Original=[518.49464347]
Predicted=[417.02762], Original=[518.47689861]
Predicted=[417.04446], Original=[511.91231167]
Predicted=[417.0575], Original=[508.04069681]
Predicted=[417.06763], Original=[506.03197208]
Predicted=[417.0756], Original=[505.73850639]
Predicted=[417.08185], Original=[505.39583111]
Predicted=[417.08667], Original=[505.85508306]
Predicted=[417.090

In [8]:
# Create a dataframe with a prediction for all datapoints for the csv

y_predicted_all = model.predict(X_shaped)
y_pred_all = np.reshape(y_predicted_all, (13608, 1))
print(y_pred_all.shape)
y_pred_all = pd.DataFrame(data=y_pred_all)
print(TimeHour.shape)
my_df = pd.DataFrame({'Time': TimeHour.loc[:,'TimeHour'].tolist(), 'pred_y': y_pred_all.loc[:,0].tolist()})
my_df.to_csv('FirstPredictions.csv')

(13608, 1)
(13608, 2)


In [9]:
df_test = pd.DataFrame(data=X)
df_test['TimeHour'] = TimeHour.loc[:, 'TimeHour']
df_test

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,TimeHour
0,0.063621,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 12:00:00
1,0.006699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 13:00:00
2,0.004883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 14:00:00
3,0.000977,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 15:00:00
4,0.027060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2018-01-01 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13607,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2019-08-25 19:00:00
13608,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
13609,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN
13610,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN


In [10]:
df_test['Before'] = 'NA'
df_test['TimeHour2'] = 'NA'
df_test['TimeHour2'] = pd.to_datetime(df_test['TimeHour'], format='%Y-%m-%d %H:%M:%S')
df_test['Diff'] = 'NA'
df_test['Modulo'] = 'NA'
for x in range(1, df_test.shape[0]):
    before = df_test.loc[x - 1, 'TimeHour2']
    now = df_test.loc[x, 'TimeHour2']
    diff = now - before
    Modulo = x%24
    df_test.loc[x, 'Diff'] = diff
    df_test.loc[x, 'Before'] = before
    df_test.loc[x, 'Modulo'] = Modulo

df_test = df_test.drop(0)
df_test['diff_hours'] = [x.total_seconds() / 3600 for x in df_test['Diff']]
df_test[df_test['diff_hours']>1]

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,TimeHour,Before,TimeHour2,Diff,Modulo,diff_hours
551,3.715820e-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2018-01-25 14:00:00,2018-01-24 10:00:00,2018-01-25 14:00:00,1 days 04:00:00,23,28.0
669,3.710938e-02,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2018-01-31 00:00:00,2018-01-30 11:00:00,2018-01-31 00:00:00,0 days 13:00:00,21,13.0
671,1.000977e-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2018-01-31 14:00:00,2018-01-31 01:00:00,2018-01-31 14:00:00,0 days 13:00:00,23,13.0
693,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2018-02-05 16:00:00,2018-02-01 11:00:00,2018-02-05 16:00:00,4 days 05:00:00,21,101.0
1068,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2018-02-23 09:00:00,2018-02-21 06:00:00,2018-02-23 09:00:00,2 days 03:00:00,12,51.0
1142,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2018-02-28 09:00:00,2018-02-26 10:00:00,2018-02-28 09:00:00,1 days 23:00:00,14,47.0
1143,2.441406e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2018-02-28 13:00:00,2018-02-28 09:00:00,2018-02-28 13:00:00,0 days 04:00:00,15,4.0
1512,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2018-04-05 20:00:00,2018-03-15 21:00:00,2018-04-05 20:00:00,20 days 23:00:00,0,503.0
1695,9.765621e-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2018-04-13 16:00:00,2018-04-13 10:00:00,2018-04-13 16:00:00,0 days 06:00:00,15,6.0
1808,3.749680e-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2018-04-18 17:00:00,2018-04-18 08:00:00,2018-04-18 17:00:00,0 days 09:00:00,8,9.0
